In [15]:
#https://contests.npcnewsonline.com/contests/2024/ifbb
#/content/drive/MyDrive/Colab Notebooks/Comets_Bodybuilding/Bodybuilding competition/Full_Size_Attempt
#/content/drive/MyDrive/Colab Notebooks/Comets_Bodybuilding/Bodybuilding_competition
URL = 'https://contests.npcnewsonline.com/contests/2024/ifbb'
URL_BASE = 'https://contests.npcnewsonline.com/contests/'
#data_path_base = '/content/drive/MyDrive/Colab Notebooks/Comets_Bodybuilding/Bodybuilding competition/'
data_path_base = '/content/drive/MyDrive/Colab Notebooks/Comets_Bodybuilding/Bodybuilding_competition/'
data_path_pkls = data_path_base + 'Pickles/'
data_path_images = data_path_base + 'Full_Size_Attempt/'
data_path_thumbnails = data_path_base + 'Thumbnails/'

In [16]:
from google.colab import drive
drive.mount('/content/drive')
#/content/drive/MyDrive/Colab Notebooks/Comets_Bodybuilding

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
pip install wget

In [18]:
# import libraries
import urllib.request
import requests

from bs4 import BeautifulSoup
import time
import pandas as pd
import calendar
import wget
import numpy as np
import os
import pickle

In [19]:
def explore_link(link):
  URL = link
  r = requests.get(URL)
  soup = BeautifulSoup(r.content, 'html5lib') # If this line causes an error, run 'pip install html5lib' or install html5lib
  print(soup.prettify())



In [20]:
def extract_contest_listing(url):
  r = requests.get(url)
  soup = BeautifulSoup(r.content, 'html5lib') # If this line causes an error, run 'pip install html5lib' or install html5lib
  contest_listing = soup.find('div', attrs = {'class': 'contest-listing'})
  #return contest_listing

  contest_listing_dict = {}

  for row in contest_listing.findAll('a'):
    contest_name = row.text.replace("\n","").replace('\t',"").replace('  ','')
    contest_listing_dict[contest_name] = row.get('href')

  return contest_listing_dict

In [21]:
def extract_divisons(contest_dict, contest_name):
  contest_link = contest_dict[contest_name]

  soup = BeautifulSoup(requests.get(contest_link).content, 'html5lib')
  #division_list = soup.find('td', attrs = {'class': "mphysique-td"})
  division_list = soup.findAll('td')
  division_dict = {}
  for division in division_list:
    label = division.text.lstrip().rstrip().split('  ')[0]
    #print("|"+label+"|")
    #print(division)
    division_dict[label] = division

  return division_dict

In [22]:
def extract_people(division_dict, division):


  people = division_dict[division].findAll('a')
  #print(people)

  df = pd.DataFrame(columns=['class', 'person_name', 'rank', 'person_link'])

  for person in people:
    row = person.text.split(' ')
    row.append(person.get('href'))
    row.append(person.get('data-parent'))
    row_filt = [item for item in row if item != '']

    #If the first entry is a number, then this is a row we want.
    if (row_filt[0] is not None) and (row_filt[0].isdigit()):
      rank = row_filt[0]
      class_name = row_filt[-1]
      person_link = row_filt[-2]
      person_name = ' '.join(row_filt[1:-2])
      df.loc[len(df.index)] = [class_name, person_name, rank, person_link]

  return df



In [23]:
#Here we want to extract the photos
from pathlib import Path

pause=False

def extract_photos(df, name, data_path, organization, contest_name, contest_date, division,debug=False):
  link = df[df['person_name'] == name]['person_link'].iloc[0]
  print("Link =", link)
  if "https://contests.npcnewsonline.com" not in link:
    print("Unexpected or nonexistent link for images for",name,"skipping!")
    return -1
  r = requests.get(link)
  soup = BeautifulSoup(r.content, 'html5lib')
  image_block = soup.find('div', attrs = {'class': "competitor-right"})
  if debug: print("Pincer 1.0")
  #now, we need to find if this page has multiple pages of images
  page_block = image_block.find('nav', attrs={'class': "pagination"})
  if type(page_block) != type(None):
    pages = page_block.findAll('li')
  else:
    print("No subpages of pictures for this one!")
    pages = [1]
  if debug: print("Pages:",pages)

  i=0
  filename_list = []
  for page in pages:
    if type(page) == int:
      pagenum = page
    else:
      pagenum = int(page.text)
    if(debug): print('page = ',page)
    if pagenum > 1:
      suffix = page.find('a').get('href')
      if (debug): print('Suffix=', suffix)
      link_page = link+suffix
      if (debug): print('Link page=', link_page)
      try:
        r = requests.get(link_page)
        soup = BeautifulSoup(r.content, 'html5lib')
        image_block = soup.find('div', attrs = {'class': "competitor-right"})
      except SSLError:
        print("SSL error on this page. Skipping!")
        continue

      #break
    print('Extracting photos from page',pagenum, "for", name)
    #print(page.prettify()
     # suffix = page.get('href')
    #print(suffix)
    # <a class="img-link" href="https://contests.npcnewsonline.com//images.php?image=15798098&amp;contest=2024-IFBB-AGP-Pro&amp;title=Yoon-Sung--Lee&amp;ids=9elw2,9elwc,9elwd,9elwe,9elwf,9elwg,9elwr,9elws,9elwt,9elwu,9elwv,9elx6,9elx7,9elx8,9elx9,9elxa,9elxl,9elxm,9elxn,9elxo,9elxp,9ely0,9ely1,9ely2,9ely3,9ely4,9elyf,9elyg,9elyh,9elyi,9elyj,9elyn,9elyo,9elyp,9elyq,9elyr,9elyx,9elyy,9elyz,9elz0,9elz1,9elz4,9elz5,9elz6,9elz7,9elz8,9elz9,9elza,9elzb,9elzc,9elzd,9elze,9elzf,9elzg,9elzh,9elzi,9elzj,9elzk,9elzl,9elzm,9elzn,9elzo,9elzp,9elzq,9elzr,9elzs,9elzt,9elzu,9elzv,9elzw,9elzx,9elzy,9elzz,9em00,9em01,9em02,9em03,9em04,9em05,9em06,9em07,9em08,9em09,9em0a,9em0b,9em0c,9em0d,9em0e,9em0f,9em0g,9em0h,9em0i,9em0j,9em0k,9em0l,9em0m,9em0n,9em0o,9em0p,9em0q,9em0r,9em0s,9em0t,9em0u,9em0v,9em0w,9em0x,9em0y,9em0z,9em10,9em11,9em12,9em13,9em14,9em15,9em16,9em17,9em18,9em19,9em1a,9em1b,9em1c,9em1d,9em1e,9em1f,9em1g,9em1h,9em1i,9em1j,9em1k,9em1l,9em1m,9em1n,9em1o,9em1p,9em1q,9em1r,9em1s,9em1t,9em1u,9em1v,9em1w,9em1x,9em1y,9em1z,9em20,9em21,9em22,9em23,9em24,9em25,9em26,9em27,9em28,9em29,9em2a,9em2b,9em2c,9em2d,9em2e,9em2f,9em2g,9em2h,9em2i,9em2j,9em2k,9em2l,9em2m,9em2n,9em2o,9em2p,9em2q,9em2r,9em2s,9em2t,9em2u,9em2v,9em2w,9em2x,9em2y,9em2z,9em30,9em31,9em32,9em33,9em34,9em35,9em36,9em37,9em38,9em39,9em3a,9em3b,9em3c,9em3d,9em3e,9em3f,9em3g,9em3h,9em3i,9em3j,9em3k,9em3l,9em3m,9em3n,9em3o,9em3p,9em3q,9em3r,9em3s" onclick="window.open('https://contests.npcnewsonline.com//images.php?image=15798098&amp;contest=2024-IFBB-AGP-Pro&amp;title=Yoon-Sung--Lee&amp;ids=9elw2,9elwc,9elwd,9elwe,9elwf,9elwg,9elwr,9elws,9elwt,9elwu,9elwv,9elx6,9elx7,9elx8,9elx9,9elxa,9elxl,9elxm,9elxn,9elxo,9elxp,9ely0,9ely1,9ely2,9ely3,9ely4,9elyf,9elyg,9elyh,9elyi,9elyj,9elyn,9elyo,9elyp,9elyq,9elyr,9elyx,9elyy,9elyz,9elz0,9elz1,9elz4,9elz5,9elz6,9elz7,9elz8,9elz9,9elza,9elzb,9elzc,9elzd,9elze,9elzf,9elzg,9elzh,9elzi,9elzj,9elzk,9elzl,9elzm,9elzn,9elzo,9elzp,9elzq,9elzr,9elzs,9elzt,9elzu,9elzv,9elzw,9elzx,9elzy,9elzz,9em00,9em01,9em02,9em03,9em04,9em05,9em06,9em07,9em08,9em09,9em0a,9em0b,9em0c,9em0d,9em0e,9em0f,9em0g,9em0h,9em0i,9em0j,9em0k,9em0l,9em0m,9em0n,9em0o,9em0p,9em0q,9em0r,9em0s,9em0t,9em0u,9em0v,9em0w,9em0x,9em0y,9em0z,9em10,9em11,9em12,9em13,9em14,9em15,9em16,9em17,9em18,9em19,9em1a,9em1b,9em1c,9em1d,9em1e,9em1f,9em1g,9em1h,9em1i,9em1j,9em1k,9em1l,9em1m,9em1n,9em1o,9em1p,9em1q,9em1r,9em1s,9em1t,9em1u,9em1v,9em1w,9em1x,9em1y,9em1z,9em20,9em21,9em22,9em23,9em24,9em25,9em26,9em27,9em28,9em29,9em2a,9em2b,9em2c,9em2d,9em2e,9em2f,9em2g,9em2h,9em2i,9em2j,9em2k,9em2l,9em2m,9em2n,9em2o,9em2p,9em2q,9em2r,9em2s,9em2t,9em2u,9em2v,9em2w,9em2x,9em2y,9em2z,9em30,9em31,9em32,9em33,9em34,9em35,9em36,9em37,9em38,9em39,9em3a,9em3b,9em3c,9em3d,9em3e,9em3f,9em3g,9em3h,9em3i,9em3j,9em3k,9em3l,9em3m,9em3n,9em3o,9em3p,9em3q,9em3r,9em3s','_blank','menubar=no,status=no,titlebar=no,toolbar=no,scrollbars=yes,width=750');return false;" target="_blank">
    for row in image_block.findAll('a'):
      if debug: print("row is ",row)
      img_tag = row.get('class')
      thumb_tag = row.find('img')
      #Thumb tag is <img loading="lazy" src="https://contests.npcnewsonline.com/images/contests/3391/thumb/16954773.jpg"/>
      if(debug): print("Thumb tag is", thumb_tag)

      thumb_url = None
      if thumb_tag is not None: thumb_url = thumb_tag.get('src')
      filename = organization + "_" + contest_name + "_" + str(contest_date) + "_" + division + "_" + df[df['person_name'] == name]['class'].values[0] + "_" + name + "_" + str(i).zfill(3) +'_thumb.jpg'

      full_filepath = os.path.join(data_path,'Thumbnails', filename)
      if thumb_url is not None:
        if not Path(full_filepath).is_file():
          if (debug):
            print("In actual production, will download thumbnail",thumb_url, "to", full_filepath)
            pass
          else:
              print("Downloading Thumbnail",i, "for", name, "from", thumb_url, "to",filename)
              wget.download(thumb_url, full_filepath)
              if pause: time.sleep(1.0 + np.random.uniform(0, 1))
        else:
          print("Skipping Thumbnail File", full_filepath, "already exists, skipping download")





      if(debug): print("Image tag is ",img_tag)
      if img_tag == ['img-link']:
        #This URL is now the url of a pop-up window with the picture in it. We still need to drill down to the actual image
        #Make sure we download the right image, not the Facebook share button!
        #http://dm6gyisv608l1.cloudfront.net/wp-content/themes/npc-2014/images/layout/fb_share_button.gif
        #https://contests.npcnewsonline.com/images/contests/3160/large/15798098.jpg
        if debug: print("Pincer 2.0")
        img_page_url = row.get('href')
        if debug: print("Pincer 3.0")
        img_soup =  BeautifulSoup(requests.get(img_page_url).content, 'html5lib')
        img_url = None
        for img_soup_sub in img_soup.findAll('img'):
          test_string = img_soup_sub.get('src')
          if ('.npcnewsonline.' in test_string) and ('.jpg' in test_string):
            img_url = test_string

        if debug: print("Pincer 4.0")
        filename = organization + "_" + contest_name + "_" + str(contest_date) + "_" + division + "_" + df[df['person_name'] == name]['class'].values[0] + "_" + name + "_" + str(i).zfill(3) +'.jpg'
        if debug: print("Filename for full image:",filename)
# Construct the full file path using os.path.join
        full_filepath = os.path.join(data_path, 'Full_Size_Attempt', filename)
        if img_url is not None:
          if not Path(full_filepath).is_file():
            if (debug):
              print("In actual production, will download ",img_url, "to", full_filepath)
              pass
            else:
              print("Downloading image ",i, "for", name, "from", img_url, "to",filename)
              wget.download(img_url, full_filepath)
              if pause: time.sleep(1.0 + np.random.uniform(0, 1))
          else:
             print("Skipping Fullsize! File", full_filepath, "already exists, skipping download")

          i+=1
          filename_list.append(filename)


  return filename_list

In [24]:
#Here we want to extract the photos
from pathlib import Path
import concurrent.futures

workers = 10

pause=False

def download_image(url, filename):
  wget.download(url, filename)

def extract_photos_parallel(df, name, data_path, organization, contest_name, contest_date, division,debug=False):
  link = df[df['person_name'] == name]['person_link'].iloc[0]
  print("Link =", link)
  if "https://contests.npcnewsonline.com" not in link:
    print("Unexpected or nonexistent link for images for",name,"skipping!")
    return -1
  r = requests.get(link)
  soup = BeautifulSoup(r.content, 'html5lib')
  image_block = soup.find('div', attrs = {'class': "competitor-right"})
  if debug: print("Pincer 1.0")
  #now, we need to find if this page has multiple pages of images
  page_block = image_block.find('nav', attrs={'class': "pagination"})
  if type(page_block) != type(None):
    pages = page_block.findAll('li')
  else:
    print("No subpages of pictures for this one!")
    pages = [1]
  if debug: print("Pages:",pages)

  i=0
  filename_list = []
  image_urls = []
  filepaths = []

  thumb_urls = []
  filepaths_thumb = []

  for page in pages:
    if type(page) == int:
      pagenum = page
    else:
      pagenum = int(page.text)
    if(debug): print('page = ',page)
    if pagenum > 1:
      suffix = page.find('a').get('href')
      if (debug): print('Suffix=', suffix)
      link_page = link+suffix
      if (debug): print('Link page=', link_page)
      try:
        r = requests.get(link_page)
        soup = BeautifulSoup(r.content, 'html5lib')
        image_block = soup.find('div', attrs = {'class': "competitor-right"})
      except SSLError:
        print("SSL error on this page. Skipping!")
        continue

      #break
    #print('Extracting photos from page',pagenum, "for", name)
    #print(page.prettify()
     # suffix = page.get('href')
    #print(suffix)
    # <a class="img-link" href="https://contests.npcnewsonline.com//images.php?image=15798098&amp;contest=2024-IFBB-AGP-Pro&amp;title=Yoon-Sung--Lee&amp;ids=9elw2,9elwc,9elwd,9elwe,9elwf,9elwg,9elwr,9elws,9elwt,9elwu,9elwv,9elx6,9elx7,9elx8,9elx9,9elxa,9elxl,9elxm,9elxn,9elxo,9elxp,9ely0,9ely1,9ely2,9ely3,9ely4,9elyf,9elyg,9elyh,9elyi,9elyj,9elyn,9elyo,9elyp,9elyq,9elyr,9elyx,9elyy,9elyz,9elz0,9elz1,9elz4,9elz5,9elz6,9elz7,9elz8,9elz9,9elza,9elzb,9elzc,9elzd,9elze,9elzf,9elzg,9elzh,9elzi,9elzj,9elzk,9elzl,9elzm,9elzn,9elzo,9elzp,9elzq,9elzr,9elzs,9elzt,9elzu,9elzv,9elzw,9elzx,9elzy,9elzz,9em00,9em01,9em02,9em03,9em04,9em05,9em06,9em07,9em08,9em09,9em0a,9em0b,9em0c,9em0d,9em0e,9em0f,9em0g,9em0h,9em0i,9em0j,9em0k,9em0l,9em0m,9em0n,9em0o,9em0p,9em0q,9em0r,9em0s,9em0t,9em0u,9em0v,9em0w,9em0x,9em0y,9em0z,9em10,9em11,9em12,9em13,9em14,9em15,9em16,9em17,9em18,9em19,9em1a,9em1b,9em1c,9em1d,9em1e,9em1f,9em1g,9em1h,9em1i,9em1j,9em1k,9em1l,9em1m,9em1n,9em1o,9em1p,9em1q,9em1r,9em1s,9em1t,9em1u,9em1v,9em1w,9em1x,9em1y,9em1z,9em20,9em21,9em22,9em23,9em24,9em25,9em26,9em27,9em28,9em29,9em2a,9em2b,9em2c,9em2d,9em2e,9em2f,9em2g,9em2h,9em2i,9em2j,9em2k,9em2l,9em2m,9em2n,9em2o,9em2p,9em2q,9em2r,9em2s,9em2t,9em2u,9em2v,9em2w,9em2x,9em2y,9em2z,9em30,9em31,9em32,9em33,9em34,9em35,9em36,9em37,9em38,9em39,9em3a,9em3b,9em3c,9em3d,9em3e,9em3f,9em3g,9em3h,9em3i,9em3j,9em3k,9em3l,9em3m,9em3n,9em3o,9em3p,9em3q,9em3r,9em3s" onclick="window.open('https://contests.npcnewsonline.com//images.php?image=15798098&amp;contest=2024-IFBB-AGP-Pro&amp;title=Yoon-Sung--Lee&amp;ids=9elw2,9elwc,9elwd,9elwe,9elwf,9elwg,9elwr,9elws,9elwt,9elwu,9elwv,9elx6,9elx7,9elx8,9elx9,9elxa,9elxl,9elxm,9elxn,9elxo,9elxp,9ely0,9ely1,9ely2,9ely3,9ely4,9elyf,9elyg,9elyh,9elyi,9elyj,9elyn,9elyo,9elyp,9elyq,9elyr,9elyx,9elyy,9elyz,9elz0,9elz1,9elz4,9elz5,9elz6,9elz7,9elz8,9elz9,9elza,9elzb,9elzc,9elzd,9elze,9elzf,9elzg,9elzh,9elzi,9elzj,9elzk,9elzl,9elzm,9elzn,9elzo,9elzp,9elzq,9elzr,9elzs,9elzt,9elzu,9elzv,9elzw,9elzx,9elzy,9elzz,9em00,9em01,9em02,9em03,9em04,9em05,9em06,9em07,9em08,9em09,9em0a,9em0b,9em0c,9em0d,9em0e,9em0f,9em0g,9em0h,9em0i,9em0j,9em0k,9em0l,9em0m,9em0n,9em0o,9em0p,9em0q,9em0r,9em0s,9em0t,9em0u,9em0v,9em0w,9em0x,9em0y,9em0z,9em10,9em11,9em12,9em13,9em14,9em15,9em16,9em17,9em18,9em19,9em1a,9em1b,9em1c,9em1d,9em1e,9em1f,9em1g,9em1h,9em1i,9em1j,9em1k,9em1l,9em1m,9em1n,9em1o,9em1p,9em1q,9em1r,9em1s,9em1t,9em1u,9em1v,9em1w,9em1x,9em1y,9em1z,9em20,9em21,9em22,9em23,9em24,9em25,9em26,9em27,9em28,9em29,9em2a,9em2b,9em2c,9em2d,9em2e,9em2f,9em2g,9em2h,9em2i,9em2j,9em2k,9em2l,9em2m,9em2n,9em2o,9em2p,9em2q,9em2r,9em2s,9em2t,9em2u,9em2v,9em2w,9em2x,9em2y,9em2z,9em30,9em31,9em32,9em33,9em34,9em35,9em36,9em37,9em38,9em39,9em3a,9em3b,9em3c,9em3d,9em3e,9em3f,9em3g,9em3h,9em3i,9em3j,9em3k,9em3l,9em3m,9em3n,9em3o,9em3p,9em3q,9em3r,9em3s','_blank','menubar=no,status=no,titlebar=no,toolbar=no,scrollbars=yes,width=750');return false;" target="_blank">
    for row in image_block.findAll('a'):
      if debug: print("row is ",row)
      img_tag = row.get('class')
      thumb_tag = row.find('img')
      #Thumb tag is <img loading="lazy" src="https://contests.npcnewsonline.com/images/contests/3391/thumb/16954773.jpg"/>
      if(debug): print("Thumb tag is", thumb_tag)

      thumb_url = None
      if thumb_tag is not None: thumb_url = thumb_tag.get('src')
      filename = organization + "_" + contest_name + "_" + str(contest_date) + "_" + division + "_" + df[df['person_name'] == name]['class'].values[0] + "_" + name + "_" + str(i).zfill(3) +'_thumb.jpg'

      full_filepath = os.path.join(data_path,'Thumbnails', filename)
      if thumb_url is not None:
        if not Path(full_filepath).is_file():
          if (debug):
            print("In actual production, will download thumbnail",thumb_url, "to", full_filepath)
            pass
          else:
            #print("Downloading Thumbnail",i, "for", name, "from", thumb_url, "to",filename)
            #wget.download(thumb_url, full_filepath)
            thumb_urls.append(thumb_url)
            filepaths_thumb.append(full_filepath)

            if pause: time.sleep(1.0 + np.random.uniform(0, 1))
        else:
          print("Skipping Thumbnail File", full_filepath, "already exists, skipping download")





      if(debug): print("Image tag is ",img_tag)
      if img_tag == ['img-link']:
        #This URL is now the url of a pop-up window with the picture in it. We still need to drill down to the actual image
        #Make sure we download the right image, not the Facebook share button!
        #http://dm6gyisv608l1.cloudfront.net/wp-content/themes/npc-2014/images/layout/fb_share_button.gif
        #https://contests.npcnewsonline.com/images/contests/3160/large/15798098.jpg
        if debug: print("Pincer 2.0")
        img_page_url = row.get('href')
        if debug: print("Pincer 3.0")
        img_soup =  BeautifulSoup(requests.get(img_page_url).content, 'html5lib')
        img_url = None
        for img_soup_sub in img_soup.findAll('img'):
          test_string = img_soup_sub.get('src')
          if ('.npcnewsonline.' in test_string) and ('.jpg' in test_string):
            img_url = test_string

        if debug: print("Pincer 4.0")
        filename = organization + "_" + contest_name + "_" + str(contest_date) + "_" + division + "_" + df[df['person_name'] == name]['class'].values[0] + "_" + name + "_" + str(i).zfill(3) +'.jpg'
        if debug: print("Filename for full image:",filename)
# Construct the full file path using os.path.join
        full_filepath = os.path.join(data_path, 'Full_Size_Attempt', filename)
        if img_url is not None:
          if not Path(full_filepath).is_file():
            if (debug):
              print("In actual production, will download ",img_url, "to", full_filepath)
              pass
            else:
              #print("Downloading image ",i, "for", name, "from", img_url, "to",filename)
              #wget.download(img_url, full_filepath)
              image_urls.append(img_url)
              filepaths.append(full_filepath)
              if pause: time.sleep(1.0 + np.random.uniform(0, 1))
          else:
             print("Skipping Fullsize! File", full_filepath, "already exists, skipping download")

          i+=1
          filename_list.append(filename)

  print("Beginning thumbnail downloads")
  with concurrent.futures.ThreadPoolExecutor(max_workers=workers) as executor:  # Adjust max_workers as needed
    futures = [executor.submit(download_image, thumb_url, full_filepath) for thumb_url, full_filepath in zip(thumb_urls, filepaths_thumb)]
    concurrent.futures.wait(futures)  # Wait for all downloads to complete

  print("Beginning full-size image downloads")

  with concurrent.futures.ThreadPoolExecutor(max_workers=workers) as executor:  # Adjust max_workers as needed
    futures = [executor.submit(download_image, img_url, full_filepath) for img_url, full_filepath in zip(image_urls, filepaths)]
    concurrent.futures.wait(futures)  # Wait for all downloads to complete

  return filename_list

In [25]:
def one_competition(contest_listing_dict,competition_name,columns,debug):
  divisions = extract_divisons(contest_listing_dict, competition_name)
  df = pd.DataFrame(columns=columns)
  #for division_name, division_code in divisions.items():
  #Identify and extract the classes and people
  if "MEN'S PHYSIQUE" in divisions:
    df_people = extract_people(divisions, "MEN'S PHYSIQUE")
    if "open" in df_people['class'].values.tolist():
      df_people_open=df_people[df_people['class']=='open']
      for index, row in df_people_open.iterrows():
        class_name = row['class']
        person_name = row['person_name']
        rank = row['rank']
        person_link = row['person_link']
        #def extract_photos(df, name, data_path, organization, contest_name, contest_date, division)
        print("Extracting photos for",person_name, "in", competition_name)
        photo_filenames = extract_photos_parallel(df_people_open, person_name, data_path_base, organization, competition_name, year, "MEN'S PHYSIQUE",debug=debug)
        print("Finished extracting photos for",person_name, "in", competition_name)
        if photo_filenames != -1:
          df.loc[len(df.index)] = [organization, competition_name, year, "MEN'S PHYSIQUE", class_name, person_name, rank, photo_filenames]

  return df

Propsed work flow to allow for dowloading without needing to download everything at once:

1.   Create a list of all competitions, probably a list of lists with each sublist being [org, competition_name]. Store this in a file
2.   For each competition, download photos and create pickle file. Store pickles and photos in separate directories
3.   After successfull download, remove that competition from the list and store as a new file
4.   Loop over steps 2-3 until we have downloaded all we need (or my Google Drive is full)



In [ ]:
#Step 1



organizations = ['ifbb','npc','npcw','cpa']
year = 2024
competition_list = []
#expand date to full yyyy-mm-dd later
for organization in organizations:
  URL = URL_BASE + str(year) + '/' + organization
  contest_listing_dict = extract_contest_listing(URL)
  #print(contest_listing_dict)
  for competition_name, competition_link in contest_listing_dict.items():
    #print(organization, ":", competition_name)
    competition_list.append([organization, competition_name])

print(competition_list)
with open(data_path_pkls+'competition_list_full.pkl', 'wb') as f:
    pickle.dump(competition_list, f)

#DO NOT UNCOMMENT THE CODE BELOW UNLESS YOU REALLY KNOW WHAT YOU ARE DOING. IT WILL RESET OUR TRACKING OF WHAT WE HAVE DOWNLOADED!
#with open(data_path_pkls+'competition_list_remaining.pkl', 'wb') as f:
#    pickle.dump(competition_list, f)

[['ifbb', 'IFBB Adela Garcia Pro'], ['ifbb', 'IFBB AGP Bikini Pro'], ['ifbb', 'IFBB AGP Classic Physique Pro'], ['ifbb', 'IFBB AGP Pro'], ['ifbb', 'IFBB AGP Pro All Stars'], ['ifbb', 'IFBB Alina Popa Classic Pro'], ['ifbb', 'IFBB Arizona Pro'], ['ifbb', 'IFBB Body Be 1 Pro'], ['ifbb', 'IFBB Border States Pro'], ['ifbb', 'IFBB California State Pro'], ['ifbb', 'IFBB California Supplements Pro'], ['ifbb', 'IFBB Charlotte Cup Pro'], ['ifbb', 'IFBB Chattanooga Night of Champions Pro'], ['ifbb', 'IFBB Chicago Pro'], ['ifbb', 'IFBB China DMS Pro Ningjin'], ['ifbb', 'IFBB China DMS Pro Tianjin'], ['ifbb', 'IFBB Colombia Pro'], ['ifbb', 'IFBB Dallas Pro'], ['ifbb', 'IFBB Daytona Pro'], ['ifbb', 'IFBB DC Pro'], ['ifbb', 'IFBB Dennis James Classic Pro'], ['ifbb', 'IFBB Detroit Pro'], ['ifbb', 'IFBB Dragon Physique DMS Pro'], ['ifbb', 'IFBB Dubai Pro'], ['ifbb', 'IFBB Emerald Cup Pro'], ['ifbb', 'IFBB Empro Pro'], ['ifbb', 'IFBB Euro Muscle Show Pro'], ['ifbb', 'IFBB Europa Pro'], ['ifbb', 'IFBB E

In [26]:
#Step 2-4
import datetime
debug=False
year = 2024

with open(data_path_pkls+'competition_list_remaining.pkl', 'rb') as f:
    competition_list = pickle.load(f)

print(competition_list)

column_names = ['organization', 'competition_name', 'year', 'division', 'class', 'person_name', 'rank', 'photo_filenames']

#Step 4

#Added [:] to solve the problem of the loop skipping items.
#The loop now iterates over this copy, leaving the original list untouched.
#You can safely remove items from the original competition_list inside the loop without affecting the iteration process.
for item in competition_list[:]:
  if debug: print(item)
  organization = item[0]
  competition_name = item[1]
  start_time = datetime.datetime.now()
  print("Starting", competition_name, "at", start_time)

  URL = URL_BASE + str(year) + '/' + organization
  if debug: print("URL=", URL)
  contest_listing_dict = extract_contest_listing(URL)
  #Step 2
  df = one_competition(contest_listing_dict,competition_name,column_names,debug)
  filename = organization + "_" + competition_name + "_" + str(year) + ".pkl"
  #Only save a pickle file if the competition actually contains photos we want
  if (df.shape[0]>0):
    df.to_pickle(os.path.join(data_path_pkls,filename))
    end_time = datetime.datetime.now()
    df['num_photos'] = df['photo_filenames'].apply(lambda x: len(x))
    total_photos = df['num_photos'].sum()
    duration_s = (end_time - start_time).seconds
    print("Downloaded",total_photos,"photos in",duration_s,"seconds")
  #Step 3
  print(competition_name, "complete. Removing from list")
  competition_list.remove(item)
  with open(data_path_pkls+'competition_list_remaining.pkl', 'wb') as f:
    pickle.dump(competition_list, f)







Streaming output truncated to the last 5000 lines.
Skipping Fullsize! File /content/drive/MyDrive/Colab Notebooks/Comets_Bodybuilding/Bodybuilding_competition/Full_Size_Attempt/ifbb_IFBB Tokyo Pro_2024_MEN'S PHYSIQUE_open_Harvey Devanadera_033.jpg already exists, skipping download
Skipping Thumbnail File /content/drive/MyDrive/Colab Notebooks/Comets_Bodybuilding/Bodybuilding_competition/Thumbnails/ifbb_IFBB Tokyo Pro_2024_MEN'S PHYSIQUE_open_Harvey Devanadera_034_thumb.jpg already exists, skipping download
Skipping Fullsize! File /content/drive/MyDrive/Colab Notebooks/Comets_Bodybuilding/Bodybuilding_competition/Full_Size_Attempt/ifbb_IFBB Tokyo Pro_2024_MEN'S PHYSIQUE_open_Harvey Devanadera_034.jpg already exists, skipping download
Skipping Thumbnail File /content/drive/MyDrive/Colab Notebooks/Comets_Bodybuilding/Bodybuilding_competition/Thumbnails/ifbb_IFBB Tokyo Pro_2024_MEN'S PHYSIQUE_open_Harvey Devanadera_035_thumb.jpg already exists, skipping download
Skipping Fullsize! File /co

In [27]:
with open(data_path_pkls+'competition_list_remaining.pkl', 'rb') as f:
    competition_list_remaining = pickle.load(f)

with open(data_path_pkls+'competition_list_full.pkl', 'rb') as f:
    competition_list_full = pickle.load(f)

print("Total Competitions Checked:",len(competition_list_full))
print("Remaining Competitions:",len(competition_list_remaining))


print(competition_list_remaining)


Total Competitions Checked: 634
Remaining Competitions: 0
[]
